# Routing

Routing allows you to define routes between component ports


# Routing bundles of ports

**Problem**: when connecting lots of ports there are collisions if we use a single port routing function such as `connect strip`

In [ ]:
import pp

xs_top = [0, 10, 20, 40, 50, 80]
pitch = 127.0
N = len(xs_top)
xs_bottom = [(i - N / 2) * pitch for i in range(N)]

top_ports = [pp.Port("top_{}".format(i), (xs_top[i], 0), 0.5, 270) for i in range(N)]

bottom_ports = [
    pp.Port("bottom_{}".format(i), (xs_bottom[i], -400), 0.5, 90) for i in range(N)
]

top_cell = pp.Component(name="connect_bundle")

for p1, p2 in zip(top_ports, bottom_ports):
    r = pp.routing.connect_strip(p1, p2)
    top_cell.add(r)
top_cell.name = "connect_bundle"
pp.qp(top_cell)
pp.show(top_cell)

**solution**: river routing routes several ports without collisions

In [ ]:
xs_top = [0, 10, 20, 40, 50, 80]
pitch = 127.0
N = len(xs_top)
xs_bottom = [(i - N / 2) * pitch for i in range(N)]

top_ports = [pp.Port("top_{}".format(i), (xs_top[i], 0), 0.5, 270) for i in range(N)]

bottom_ports = [
    pp.Port("bottom_{}".format(i), (xs_bottom[i], -400), 0.5, 90) for i in range(N)
]

top_cell = pp.Component(name="connect_bundle")
elements = pp.routing.connect_bundle(top_ports, bottom_ports)
for e in elements:
    top_cell.add(e)
top_cell.name = "connect_bundle"
pp.qp(top_cell)
pp.show(top_cell)

In [ ]:
xs_top = [0, 10, 20, 40, 50, 80]
pitch = 127.0
N = len(xs_top)
xs_bottom = [(i - N / 2) * pitch for i in range(N)]

top_ports = [pp.Port("top_{}".format(i), (xs_top[i], 0), 0.5, 270) for i in range(N)]

bottom_ports = [
    pp.Port("bottom_{}".format(i), (xs_bottom[i], -400), 0.5, 90) for i in range(N)
]

top_cell = pp.Component(name="connect_bundle")
elements = pp.routing.link_optical_ports(top_ports, bottom_ports)
for e in elements:
    top_cell.add(e)
top_cell.name = "link_optical_ports"
pp.qp(top_cell)
pp.show(top_cell)

# Routing banks of ports through pre-defined waypoints

In [ ]:
import numpy as np
import pp
from pp.routing.connect_bundle_from_waypoints import connect_bundle_waypoints


@pp.autoname
def test_connect_bundle_waypoints():
    import pp
    from pp.component import Port

    xs1 = np.arange(10) * 5 - 500.0

    N = xs1.size
    ys2 = np.array([0, 5, 10, 20, 25, 30, 40, 55, 60, 75]) + 500.0

    ports1 = [Port("A_{}".format(i), (xs1[i], 0), 0.5, 90) for i in range(N)]
    ports2 = [Port("B_{}".format(i), (0, ys2[i]), 0.5, 180) for i in range(N)]

    top_cell = pp.Component()
    way_points = [
        ports1[0].position,
        ports1[0].position + (0, 100),
        ports1[0].position + (200, 100),
        ports1[0].position + (200, -200),
        ports1[0].position + (0, -200),
        ports1[0].position + (0, -350),
        ports1[0].position + (400, -350),
        (ports1[0].x + 400, ports2[-1].y),
        ports2[-1].position,
    ]

    elements = connect_bundle_waypoints(ports1, ports2, way_points)
    top_cell.add(elements)

    return top_cell

cell = test_connect_bundle_waypoints()
pp.plotgds(cell)

# Route bundles through corners

In [ ]:
from pp.name import autoname
from pp.component import Component
from pp.port import Port
from pp.routing import connect_bundle

@pp.autoname
def test_connect_corner(N=6, config="A"):

    d = 10.0

    sep = 5.0
    top_cell = pp.Component(name="connect_corner")

    if config in ["A", "B"]:
        a = 100.0
        ports_A_TR = [
            Port("A_TR_{}".format(i), (d, a / 2 + i * sep), 0.5, 0) for i in range(N)
        ]
        ports_A_TL = [
            Port("A_TL_{}".format(i), (-d, a / 2 + i * sep), 0.5, 180) for i in range(N)
        ]
        ports_A_BR = [
            Port("A_BR_{}".format(i), (d, -a / 2 - i * sep), 0.5, 0) for i in range(N)
        ]
        ports_A_BL = [
            Port("A_BL_{}".format(i), (-d, -a / 2 - i * sep), 0.5, 180)
            for i in range(N)
        ]

        ports_A = [ports_A_TR, ports_A_TL, ports_A_BR, ports_A_BL]

        ports_B_TR = [
            Port("B_TR_{}".format(i), (a / 2 + i * sep, d), 0.5, 90) for i in range(N)
        ]
        ports_B_TL = [
            Port("B_TL_{}".format(i), (-a / 2 - i * sep, d), 0.5, 90) for i in range(N)
        ]
        ports_B_BR = [
            Port("B_BR_{}".format(i), (a / 2 + i * sep, -d), 0.5, 270) for i in range(N)
        ]
        ports_B_BL = [
            Port("B_BL_{}".format(i), (-a / 2 - i * sep, -d), 0.5, 270)
            for i in range(N)
        ]

        ports_B = [ports_B_TR, ports_B_TL, ports_B_BR, ports_B_BL]

    elif config in ["C", "D"]:
        a = N * sep + 2 * d
        ports_A_TR = [
            Port("A_TR_{}".format(i), (a, d + i * sep), 0.5, 0) for i in range(N)
        ]
        ports_A_TL = [
            Port("A_TL_{}".format(i), (-a, d + i * sep), 0.5, 180) for i in range(N)
        ]
        ports_A_BR = [
            Port("A_BR_{}".format(i), (a, -d - i * sep), 0.5, 0) for i in range(N)
        ]
        ports_A_BL = [
            Port("A_BL_{}".format(i), (-a, -d - i * sep), 0.5, 180) for i in range(N)
        ]

        ports_A = [ports_A_TR, ports_A_TL, ports_A_BR, ports_A_BL]

        ports_B_TR = [
            Port("B_TR_{}".format(i), (d + i * sep, a), 0.5, 90) for i in range(N)
        ]
        ports_B_TL = [
            Port("B_TL_{}".format(i), (-d - i * sep, a), 0.5, 90) for i in range(N)
        ]
        ports_B_BR = [
            Port("B_BR_{}".format(i), (d + i * sep, -a), 0.5, 270) for i in range(N)
        ]
        ports_B_BL = [
            Port("B_BL_{}".format(i), (-d - i * sep, -a), 0.5, 270) for i in range(N)
        ]

        ports_B = [ports_B_TR, ports_B_TL, ports_B_BR, ports_B_BL]

    if config in ["A", "C"]:
        for ports1, ports2 in zip(ports_A, ports_B):
            elements = connect_bundle(ports1, ports2)
            top_cell.add(elements)

    elif config in ["B", "D"]:
        for ports1, ports2 in zip(ports_A, ports_B):
            elements = connect_bundle(ports2, ports1)
            top_cell.add(elements)

    return top_cell


c = test_connect_corner()
pp.qp(c)
pp.show(c)

In [ ]:
@autoname
def test_connect_bundle_udirect(dy=200, angle=270):

    xs1 = [-100, -90, -80, -55, -35, 24, 0] + [200, 210, 240]

    axis = "X" if angle in [0, 180] else "Y"

    pitch = 10.0
    N = len(xs1)
    xs2 = [50 + i * pitch for i in range(N)]

    if axis == "X":
        ports1 = [Port("top_{}".format(i), (0, xs1[i]), 0.5, angle) for i in range(N)]

        ports2 = [
            Port("bottom_{}".format(i), (dy, xs2[i]), 0.5, angle) for i in range(N)
        ]

    else:
        ports1 = [Port("top_{}".format(i), (xs1[i], 0), 0.5, angle) for i in range(N)]

        ports2 = [
            Port("bottom_{}".format(i), (xs2[i], dy), 0.5, angle) for i in range(N)
        ]

    top_cell = Component(name="connect_bundle_udirect")
    elements = connect_bundle(ports1, ports2)
    for e in elements:
        top_cell.add(e)

    return top_cell


c = test_connect_bundle_udirect()
pp.show(c)
pp.qp(c)

In [ ]:
@autoname
def test_connect_bundle_u_indirect(dy=-200, angle=180):

    xs1 = [-100, -90, -80, -55, -35] + [200, 210, 240]

    axis = "X" if angle in [0, 180] else "Y"

    pitch = 10.0
    N = len(xs1)
    xs2 = [50 + i * pitch for i in range(N)]

    a1 = angle
    a2 = a1 + 180

    if axis == "X":
        ports1 = [Port("top_{}".format(i), (0, xs1[i]), 0.5, a1) for i in range(N)]

        ports2 = [Port("bottom_{}".format(i), (dy, xs2[i]), 0.5, a2) for i in range(N)]

    else:
        ports1 = [Port("top_{}".format(i), (xs1[i], 0), 0.5, a1) for i in range(N)]

        ports2 = [Port("bottom_{}".format(i), (xs2[i], dy), 0.5, a2) for i in range(N)]

    top_cell = Component("connect_bundle_u_indirect")
    elements = connect_bundle(ports1, ports2)
    for e in elements:
        top_cell.add(e)

    return top_cell


c = test_connect_bundle_u_indirect()
pp.show(c)
pp.qp(c)

In [ ]:
@autoname
def test_facing_ports():
    dy = 200.0
    xs1 = [-500, -300, -100, -90, -80, -55, -35, 200, 210, 240, 500, 650]

    pitch = 10.0
    N = len(xs1)
    xs2 = [-20 + i * pitch for i in range(N // 2)]
    xs2 += [400 + i * pitch for i in range(N // 2)]

    a1 = 90
    a2 = a1 + 180

    ports1 = [Port("top_{}".format(i), (xs1[i], 0), 0.5, a1) for i in range(N)]
    ports2 = [Port("bottom_{}".format(i), (xs2[i], dy), 0.5, a2) for i in range(N)]

    top_cell = Component("test_facing_ports")
    elements = connect_bundle(ports1, ports2)
    # elements = link_ports_path_length_match(ports1, ports2)
    top_cell.add(elements)

    return top_cell


c = test_facing_ports()
pp.show(c)
pp.qp(c)

In [ ]:
def demo_connect_bundle():
    """ combines all the connect_bundle tests """
    y = 400.0
    x = 500
    y0 = 900
    dy = 200.0
    c = Component("connect_bundle")
    for j, s in enumerate([-1, 1]):
        for i, angle in enumerate([0, 90, 180, 270]):
            _cmp = test_connect_bundle_u_indirect(dy=s * dy, angle=angle)
            _cmp_ref = _cmp.ref(position=(i * x, j * y))
            c.add(_cmp_ref)

            _cmp = test_connect_bundle_udirect(dy=s * dy, angle=angle)
            _cmp_ref = _cmp.ref(position=(i * x, j * y + y0))
            c.add(_cmp_ref)

    for i, config in enumerate(["A", "B", "C", "D"]):
        _cmp = test_connect_corner(config=config)
        _cmp_ref = _cmp.ref(position=(i * x, 1700))
        c.add(_cmp_ref)

    _cmp = test_facing_ports()
    _cmp_ref = _cmp.ref(position=(800, 1820))
    c.add(_cmp_ref)

    return c

c = demo_connect_bundle()
pp.show(c)
pp.qp(c)